In [ ]:
# setup

import psycopg2
import pandas as pd
from sqlalchemy import create_engine, text
import warnings

warnings.filterwarnings("ignore")

Forgot to remove stops outside berlin area (they dont have matching district ids)

In [34]:
# import csv

bus_tram_stops_test = pd.read_csv('/Users/bastianlenkers/VSCode_Stuff/Repos/layered-populate-data-pool-da/tram_bus_data_modelling/bus_trams/sources/bus_tram_stops.csv')

In [ ]:
# drop ambigious index column

bus_tram_stops_test.drop('index', axis=1, inplace=True)

In [ ]:
# forgot to filter stations outside berlin area... 

bus_tram_stops = bus_tram_stops_test[bus_tram_stops['district_id'] != 'unknown']

Now its business time

In [ ]:
# setup credentials

username = 'sebastian_nichtweiss'
password = '0hoZh9X7uJxnie'

In [24]:
# Conection
host = 'localhost'
port = '5433'
database = 'layereddb'
schema='berlin_source_data'

#connection to db
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')

In [28]:
# create query

create_table_query = f"""
CREATE TABLE IF NOT EXISTS {schema}.bus_tram_stops (
    stop_id INTEGER PRIMARY KEY,
    district_id VARCHAR(100),
    name VARCHAR(200),
    address VARCHAR(200),
    latitude DECIMAL(9,6),
    longitude DECIMAL(9,6),
    neighborhood VARCHAR(100),
    district VARCHAR(100),
    CONSTRAINT fk_district_id FOREIGN KEY (district_id)
        REFERENCES berlin_source_data.districts(district_id)
        ON DELETE RESTRICT
        ON UPDATE CASCADE
);
"""

In [29]:
# Execute the query to create empty table
with engine.connect() as conn:
    conn.execute(text(create_table_query))
    conn.commit()  # commit the transaction

In [38]:
#  Send the DataFrame to the database using .to_sql()
bus_tram_stops.to_sql(
    f'bus_tram_stops',        # table name
    engine,
    schema=schema,
    if_exists='append', # replace table if it exists
    index=False
)

print("DataFrame sent to PostgreSQL using .to_sql() with psycopg2!")

DataFrame sent to PostgreSQL using .to_sql() with psycopg2!
